In [1]:
from scipy.io import wavfile
import IPython
import numpy as np
import ffmpy
import wave
import numpy as np

In [48]:
rate1, data1 = wavfile.read("music_quieter.wav")
rate2, data2 = wavfile.read("../adversarial_asr/LibriSpeech/test-clean/2961/960/2961-960-0020.wav")

In [33]:
IPython.display.Audio(data=data1, rate=rate1)

In [4]:
IPython.display.Audio(data=data2, rate=rate2)

## Volume 

In [7]:
from pydub import AudioSegment
from pydub.playback import play

song = AudioSegment.from_wav("../adversarial_asr/LibriSpeech/test-clean/2961/960/2961-960-0020.wav")
song_50_db_quieter = song - 50
# play(song_10_db_quieter)
song_50_db_quieter.export("results/quieter.wav", "wav")

<_io.BufferedRandom name='results/quieter.wav'>

In [9]:
song = AudioSegment.from_wav("../adversarial_asr/LibriSpeech/test-clean/2961/960/2961-960-0020.wav")
song_50_db_louder = song + 50
# play(song_50_db_louder)
song_50_db_louder.export("results/louder.wav", "wav")

<_io.BufferedRandom name='results/louder.wav'>

## Speed

In [11]:
#0.5 TIMES SLOWER
ff = ffmpy.FFmpeg(inputs={"../adversarial_asr/LibriSpeech/test-clean/2961/960/2961-960-0020.wav": None}, outputs={"results/slower.wav": ["-filter:a", "atempo=0.5"]})
ff.run()

(None, None)

In [12]:
#1.5 TIMES FASTER
ff = ffmpy.FFmpeg(inputs={"../adversarial_asr/LibriSpeech/test-clean/2961/960/2961-960-0020.wav": None}, outputs={"results/faster.wav": ["-filter:a", "atempo=1.5"]})
ff.run()

(None, None)

## Noise

In [13]:
def fftnoise(f):
    f = np.array(f, dtype="complex")
    Np = (len(f) - 1) // 2
    phases = np.random.rand(Np) * 2 * np.pi
    phases = np.cos(phases) + 1j * np.sin(phases)
    f[1 : Np + 1] *= phases
    f[-1 : -1 - Np : -1] = np.conj(f[1 : Np + 1])
    return np.fft.ifft(f).real

def band_limited_noise(min_freq, max_freq, samples=1024, samplerate=1):
    freqs = np.abs(np.fft.fftfreq(samples, 1 / samplerate))
    f = np.zeros(samples)
    f[np.logical_and(freqs >= min_freq, freqs <= max_freq)] = 1
    return fftnoise(f)

In [25]:
noise = band_limited_noise(min_freq=1000, max_freq=20000, samples=len(data2), samplerate=rate2) * 10
audio_clip_band_limited = data2 + noise

In [26]:
IPython.display.Audio(data=audio_clip_band_limited, rate=rate2)

In [29]:
audio_clip_band_limited = np.asarray(audio_clip_band_limited, dtype=np.int16)
wavfile.write("results/band_limited.wav", rate2, audio_clip_band_limited)

In [30]:
noise = np.asarray(noise, dtype=np.int16)
wavfile.write("results/noise.wav", rate2, noise)

In [47]:
music = AudioSegment.from_wav("music.wav")
music_50_db_quieter = music - 30
play(music_50_db_quieter)
music_50_db_quieter.export("music_quieter.wav", "wav")

<_io.BufferedRandom name='music_quieter.wav'>

In [50]:
audio_music = data1[:len(data2)] + data2
wavfile.write("results/music.wav", rate2, audio_music)

In [51]:
IPython.display.Audio(data=audio_music, rate=rate1)

## Frequency 

In [17]:
fr = 20
shift = 500//fr  # shifting 25 Hz

wr = wave.open('../adversarial_asr/LibriSpeech/test-clean/2961/960/2961-960-0020.wav', 'r')
# Set the parameters for the output file.
par = list(wr.getparams())
par[3] = 0  # The number of samples will be set by writeframes.
par = tuple(par)
ww = wave.open('results/freq.wav', 'w')
ww.setparams(par)

sz = wr.getframerate()//fr  # Read and process 1/fr second at a time.
# A larger number for fr means less reverb.
c = int(wr.getnframes()/sz)  # count of the whole file
for num in range(c):
    da = np.fromstring(wr.readframes(sz), dtype=np.int16)
    left, right = da[0::2], da[1::2]  # left and right channel
    lf, rf = np.fft.rfft(left), np.fft.rfft(right)
    lf, rf = np.roll(lf, shift), np.roll(rf, shift)
    lf[0:shift], rf[0:shift] = 0, 0
    nl, nr = np.fft.irfft(lf), np.fft.irfft(rf)
    ns = np.column_stack((nl, nr)).ravel().astype(np.int16)
    ww.writeframes(ns.tostring())
    
wr.close()
ww.close()

/Users/xinyi/opt/anaconda3/envs/cloudml/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  app.launch_new_instance()


## Pitch

In [20]:
import librosa
y, sr = librosa.load('../adversarial_asr/LibriSpeech/test-clean/2961/960/2961-960-0020.wav', sr=16000) # y is a numpy array of the wav file, sr = sample rate
y_shifted = librosa.effects.pitch_shift(y, sr, n_steps=4)
librosa.output.write_wav("results/pitch.wav", y_shifted, sr)